In [1]:
import numpy as np

In [37]:
# Tworzenie macierzy dwudiagonalnej:

np.random.seed(0)
q = np.random.rand(5) # główna diagonala
r = np.random.rand(4) # naddiagonala
B1 = np.diag(q)
B2 = np.diag(r, k=1)
B = B1 + B2
B

array([[0.5488135 , 0.64589411, 0.        , 0.        , 0.        ],
       [0.        , 0.71518937, 0.43758721, 0.        , 0.        ],
       [0.        , 0.        , 0.60276338, 0.891773  , 0.        ],
       [0.        , 0.        , 0.        , 0.54488318, 0.96366276],
       [0.        , 0.        , 0.        , 0.        , 0.4236548 ]])

In [56]:
# Znajdowanie podziału macierzy B na B1, B2

m = B.shape[0]//2 # wiersz podziału
B1 = B[:m, :(m+1)]
B2 = B[(m+1):, (m+1):]
qm = B[m, m]
rm = B[m, m+1]

In [57]:
B1

array([[0.5488135 , 0.64589411, 0.        ],
       [0.        , 0.71518937, 0.43758721]])

In [58]:
B2

array([[0.54488318, 0.96366276],
       [0.        , 0.4236548 ]])

In [59]:
qm, rm

(0.6027633760716439, 0.8917730007820798)

In [60]:
from scipy.linalg import svd

In [70]:
U1, S1, V1T = svd(B1)
U2, S2, V2T = svd(B2)

In [71]:
V1T

array([[-0.3613342 , -0.88834382, -0.28334228],
       [-0.77169161,  0.11433601,  0.62563514],
       [ 0.52338289, -0.44471623,  0.72684099]])

In [72]:
V2T

array([[ 0.44083771,  0.89758683],
       [-0.89758683,  0.44083771]])

In [79]:
l1 = V1T[:-1, -1]; print(qm*l1)
nu = V1T[-1, -1]; print(nu)

f2 = V2T[:, 0]; print(rm*f2)

z = np.hstack((nu, qm*l1, rm*f2)); print(z)

[-0.17078835  0.37710995]
0.7268409887895942
[ 0.39312717 -0.8004437 ]
[ 0.72684099 -0.17078835  0.37710995  0.39312717 -0.8004437 ]


In [92]:
d = np.hstack((0, S1, S2)); print(d)
D = np.diag(d); print(D)

[0.         1.08290243 0.49867443 1.16876733 0.19750926]
[[0.         0.         0.         0.         0.        ]
 [0.         1.08290243 0.         0.         0.        ]
 [0.         0.         0.49867443 0.         0.        ]
 [0.         0.         0.         1.16876733 0.        ]
 [0.         0.         0.         0.         0.19750926]]


In [91]:
1.083*1.083

1.1728889999999998

In [374]:
# Teraz trzeba rozwiązać pełne zadanie własne macierzy D^2 + z*z^T
# W tym celu stworzymy sobie inną macierz diagonalną D oraz inny wektor z
D = np.diag([-0.11, 0.23, 0.23, 0.23, 0.38, 0.49, 1.13])
z = np.array([0.13, -2.11, 0, 3.37, 5.18, 0.92, 0])

ind = sum(z == 0)
perm = (z == 0)

N = D.shape[0]

A = D + z.reshape((N, 1)) @ z.reshape((1, N))

In [375]:
# Przypadek 1: z_i = 0
# wektor własny = e_i
# wypełniamy odpowiednio macierz U
U = np.zeros((N, N))
for i in np.where(z == 0)[0]:
    U[:, i] = np.hstack([[0]*i, 1, [0]*(N-i-1)])

# Permutujemy zgodnie z notatkami
PAPt = np.block([
    [A[z==0, :][:, z==0], A[z==0, :][:, z!=0]],
    [A[z!=0, :][:, z==0], A[z!=0, :][:, z!=0]]
]) # to jest macierz P*A*P'

z = np.hstack((z[z==0], z[z!=0])) # przepermutowany "z"
zhat = z[z!=0] # niezerowe elementy "z"

U = np.hstack((U[:, z==0], U[:, z!=0])) # w U kolumnowo idą tylko wektory własne, więc zamieniamy tylko kolumny

In [376]:
# P = np.diag([1]*N)
# P = np.vstack((P[z==0], P[z!=0]))
# PAP_T = P @ A @ P.T

# Ahat = PAP_T[ind:, ind:]; print(Ahat)

In [377]:
# Teraz należy znaleźć rozkład spektralny macierzy:
Ahat = PAPt[ind:, ind:]; Ahat

array([[ -0.0931,  -0.2743,   0.4381,   0.6734,   0.1196],
       [ -0.2743,   4.6821,  -7.1107, -10.9298,  -1.9412],
       [  0.4381,  -7.1107,  11.5869,  17.4566,   3.1004],
       [  0.6734, -10.9298,  17.4566,  27.2124,   4.7656],
       [  0.1196,  -1.9412,   3.1004,   4.7656,   1.3364]])

In [378]:
D1 = np.diag(D[perm, perm]); D1

array([[0.23, 0.  ],
       [0.  , 1.13]])

In [379]:
Dhat = np.diag(D[~perm, ~perm]); Dhat

array([[-0.11,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.23,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.38,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.49]])

In [380]:
# ta macierz powinna być równa D1
Ahat - zhat.reshape((N-ind, 1)) @ zhat.reshape((1, N-ind))
# i jest!

array([[-0.11,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.23,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.38,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.49]])

In [381]:
# Przypadek 2: 
def count_elements(arr: np.array) -> dict:
    
    retdict = {}
    for i, a in enumerate(arr):
        if retdict.get(a) is None:
            retdict[a] = [i]
        else:
            retdict[a].append(i)
    return retdict

ce = count_elements(np.diag(D))

In [382]:
Mhat, Nhat = Ahat.shape
Ahat

array([[ -0.0931,  -0.2743,   0.4381,   0.6734,   0.1196],
       [ -0.2743,   4.6821,  -7.1107, -10.9298,  -1.9412],
       [  0.4381,  -7.1107,  11.5869,  17.4566,   3.1004],
       [  0.6734, -10.9298,  17.4566,  27.2124,   4.7656],
       [  0.1196,  -1.9412,   3.1004,   4.7656,   1.3364]])

In [417]:
for key, val in ce.items():
    if len(val) > 1:
        print(f"{key} jest {len(val)-1}-krotną wartością własną Ahat.")
        i = val[0]
        k = len(val)
        temp_zhat = zhat[i:(i+k)]
        
        u = np.copy(temp_zhat)
        u[-1] += np.linalg.norm(u)
        v = u/np.linalg.norm(u)
        Hi = np.eye(k) - 2 * v.reshape((k, 1)) @ v.reshape((1, k))

0.23 jest 2-krotną wartością własną Ahat.


In [423]:
z

array([ 0.  ,  0.  ,  0.13, -2.11,  3.37,  5.18,  0.92])

In [411]:
H = np.block([
    [np.eye(ind+i),               np.zeros((ind+i, k)),         np.zeros((ind+i, Nhat-i-k))],
    [np.zeros((k, ind+i)),        Hi,                       np.zeros((k, Nhat-i-k))],
    [np.zeros((Nhat-i-k, ind+i)), np.zeros((Nhat-i-k, k)),  np.eye(Nhat-i-k)]
    ])

In [412]:
H.shape

(7, 7)

In [413]:
D

array([[-0.11,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.23,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.23,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.23,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.38,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.49,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.13]])

In [406]:
x = H @ A @ H.T

In [409]:
np.isclose(D + ztilde.reshape((N, 1)) @ ztilde.reshape((1, N)), x)

array([[False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False]])